# Equity Linked Notes - Autocallable Phoenix (single)

This notebook demonstrates how to access and use the functionalities of **Autocallable Structured Notes** which are part of our **QPS** module within LSEG Financial Analytics SDK. This notebook presents a Phoenix structure with single underlying.

**You will be able to:**
* Define Phoenix instrument with basic parameters
* Configure pricing parameters
* Evaluate Phoenix Analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp 

import json
import datetime as dt
import pandas as pd
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example Autocallable Phoenix, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (strike, dates, notional, index)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing parameters, optional

In [2]:
# 1. Create SP definition object

SN_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "Phoenix",
    inputs = [
        sp.NameTypeValue(name="notional", type = "string", value="1000"),
        sp.NameTypeValue(name="Underlying", type = "string", value="BNPP_PA"),
        sp.NameTypeValue(name="StrikeDate", type = "date", value="16/07/2025"),
        sp.NameTypeValue(name="MaturityDate", type = "date", value="23/07/2029"),
        sp.NameTypeValue(name="LastValuationDate", type = "date", value="16/07/2029"),
        sp.NameTypeValue(name="Schedule", type = "schedule", value=[
                    ["16/10/2025", "16/10/2025", "23/10/2025", "23/10/2025"],
                    ["16/01/2026", "16/01/2026", "23/01/2026", "23/01/2026"],
                    ["16/04/2026", "16/04/2026", "23/04/2026", "23/04/2026"],
                    ["16/07/2026", "16/07/2026", "23/07/2026", "23/07/2026"],
                    ["16/10/2026", "16/10/2026", "23/10/2026", "23/10/2026"],
                    ["18/01/2027", "18/01/2027", "25/01/2027", "25/01/2027"],
                    ["16/04/2027", "16/04/2027", "23/04/2027", "23/04/2027"],
                    ["16/07/2027", "16/07/2027", "23/07/2027", "23/07/2027"],
                    ["18/10/2027", "18/10/2027", "25/10/2027", "25/10/2027"],
                    ["17/01/2028", "17/01/2028", "24/01/2028", "24/01/2028"],
                    ["17/04/2028", "17/04/2028", "24/04/2028", "24/04/2028"],
                    ["17/07/2028", "17/07/2028", "24/07/2028", "24/07/2028"],
                    ["16/10/2028", "16/10/2028", "23/10/2028", "23/10/2028"],
                    ["16/01/2029", "16/01/2029", "23/01/2029", "23/01/2029"],
                    ["16/04/2029", "16/04/2029", "23/04/2029", "23/04/2029"]]),
        sp.NameTypeValue(name="CouponBarrier", type = "curve", value=[["16/07/2025","80%"]]),
        sp.NameTypeValue(name="FinalCouponBarrier", type = "string", value="60%"),
        sp.NameTypeValue(name="AutocallBarrier", type = "curve", value=[["16/07/2026","100%"]]),
        sp.NameTypeValue(name="RedemptionBarrier", type = "string", value="60%"),
        sp.NameTypeValue(name="CouponRate", type = "string", value="1.75%"),
        sp.NameTypeValue(name="Leverage", type = "string", value="100%"),
        sp.NameTypeValue(name="VarInit", type = "string", value="3")
    ],
    payoff_description = [
          [
            "Schedule type",
            "Schedule description",
            "Index",
            "Performance",
            "Alive",
            "Early",
            "Count",
            "Coupon",
            "Settlement",
            "OptionAtMaturity",
            "PriceIn%",
            "Price"
          ],
          [
            "AtDate",
            "StrikeDate",
            "EqSpot(Underlying)",
            "",
            "1",
            "",
            "$n=VarInit;$n",
            "",
            "",
            "",
            "",
            ""
          ],
          [
            "OnUserSchedule",
            "Schedule",
            "EqSpot(Underlying)",
            "Index[t] / Index[StrikeDate]",
            "If(Performance[t] >= Interpol(AutocallBarrier,PS()), 0, Alive[LastDate])",
            "(1 - Alive[t]) * Alive[LastDate(-1)]",
            "$n=$n+1;$n",
            "Receive $n*CouponRate * Early[t]",
            "Receive Early[t]",
            "",
            "",
            ""
          ],
          [
            "AtDate",
            "LastValuationDate",
            "EqSpot(Underlying)",
            "Index[t] / Index[StrikeDate]",
            "",
            "",
            "$n=$n+1;$n",
            "Receive (MaturityDate, If(Performance[t]>=FinalCouponBarrier, $n*CouponRate, 0) * Alive[LastDate])",
            "Receive (MaturityDate, Alive[LastDate])",
            "Receive (MaturityDate, If(Performance[t]>=RedemptionBarrier,0,(Performance[t] / Leverage -1)*Alive[LastDate]))",
            "Report((columnval(Coupon)+columnval(OptionAtMaturity)+columnval(Settlement))*100)",
            "Report(columnval(PriceIn%)/100*Notional)"
          ]
        ]
)


# 2. Create SP instrument definition object

phoenix = sp.StructuredProductsDefinitionInstrument(definition = SN_definition)
print("Instrument definition created")


# 3. Create SP parameters object - optional

phoenix_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2025, 8, 20),  # Set your desired valuation date
    report_ccy="USD",  # Set your reporting currency
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
                  underlying_code = "BNPP.PA",
                  underlying_name = "BNPP_PA",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire")
          ]
)

print("Pricing parameters configured")

Instrument definition created
Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing
try:
    response = sp.price(
        definitions=[phoenix],
        pricing_preferences=phoenix_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )
    
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing execution completed")
    
except Exception as e:
    print(f"Price calculation failed: {str(e)}")
    raise

Pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., strike, dates, notional, underlying, barrier, profit target), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` (payments, fixings)
   - **description**: Instrument summary StructuredProductDefinition and also the default fields not specified in the StructuredProductDefinition, but used by default in the calculation
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

#### Description
Useful for understanding which fields are included by default in the price function, even if they are not explicitly specified.

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "Phoenix",
    "dealCcy": "EUR",
    "reportCcy": "USD",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Alive": 2.084285062,
        "Count": 175.008218382,
        "Coupon": 0.08093058,
        "Early": 0.899818091,
        "Index": 1165.803110422,
        "OptionAtMaturity": -0.048813945,
        "Performance": 15.323384732,
        "Price": 1017.948154845,
        "PriceIn%": 101.794815485,
        "Settlement": 0.985831519
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, Delta. Make sure a 'PricePercent' column is specified in payoff description to compute greeks. "
}


#### Analytics

##### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_phoenix_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_phoenix_valuation)

,Field,Value
0,marketValueInDealCcy,1017.948155
1,marketValueInReportCcy,1186.062293


##### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Extract underlyings
model_df = pd.DataFrame(data=phoenix_pricing_params.models)
underlying_list = model_df['underlyingName'].to_list()

# Build dataframes for all cash flow types
output = []
for cf_type in cashflows:
    cashflow_df = pd.DataFrame(cf_type['payments'])
    if cf_type['legTag'] == 'Index':
        cashflow_df = cashflow_df.rename(columns={'amount': underlying_list[0]})
    else:
        cashflow_df = cashflow_df.rename(columns={'amount': cf_type['legTag']})
    cashflow_df['discountFactor'] = cashflow_df['discountFactor'].round(4)
    output.append(cashflow_df)

# Merge all dataframes on the 'date' column
combined_df = pd.concat([*output],axis=1)

# Remove duplicated columns with the same values
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

common_cols = ['date', 'discountFactor', 'Performance', 'Alive', 'Early', 'Coupon', 'Settlement', 'OptionAtMaturity', 'Price', 'currency', 'occurence']
indv_perf = [f'PERF_{underlying}' for underlying in underlying_list if len(underlying_list) > 1]
cols_to_display = common_cols[0:2] + underlying_list + indv_perf + common_cols[2:]

# Leave only columns to display
combined_df = combined_df.loc[:,[*cols_to_display]]

# Display the combined dataframe
display(combined_df)

,date,discountFactor,BNPP_PA,Performance,Alive,Early,Coupon,Settlement,OptionAtMaturity,Price,currency,occurence
0,2025-07-16,1.0000,76.080000,0.000000,1.0000,0.0000,0.000000,0.000000,0.000000,0.00000,EUR,Historical
1,2025-10-16,0.9968,78.454731,1.031214,0.3740,0.6260,0.043803,0.625759,0.000000,0.00000,EUR,Future
2,2026-01-16,0.9919,78.823350,1.036059,0.2594,0.1146,0.010024,0.114558,0.000000,0.00000,EUR,Future
3,2026-04-16,0.9873,79.148273,1.040330,0.1878,0.0716,0.007515,0.071575,0.000000,0.00000,EUR,Future
4,2026-07-16,0.9828,79.554744,1.045672,0.1554,0.0324,0.003968,0.032389,0.000000,0.00000,EUR,Future
5,2026-10-16,0.9782,79.868678,1.049799,0.1350,0.0204,0.002855,0.020393,0.000000,0.00000,EUR,Future
6,2027-01-18,0.9735,80.244485,1.054738,0.1204,0.0146,0.002299,0.014595,0.000000,0.00000,EUR,Future
7,2027-04-16,0.9690,80.708290,1.060835,0.1116,0.0088,0.001539,0.008797,0.000000,0.00000,EUR,Future
8,2027-07-16,0.9642,75.583268,0.993471,0.1106,0.0010,0.000192,0.001000,0.000000,0.00000,EUR,Future
9,2027-10-18,0.9590,76.053382,0.999650,0.1078,0.0028,0.000588,0.002799,0.000000,0.00000,EUR,Future


##### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_phoenix_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_phoenix_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,7.349090
1,deltaAmountInReportCcy,8.562792
2,gammaAmountInDealCcy,-0.532039
3,gammaAmountInReportCcy,-0.619905
4,vegaAmountInDealCcy,-2.140300
5,vegaAmountInReportCcy,-2.493771
6,thetaAmountInDealCcy,-0.140434
7,thetaAmountInReportCcy,-0.163627
